In [1]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [2]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [3]:
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket ' + bucket + ' could not be created.'
    return 'Created or already exists ' + bucket + ' bucket.'

In [4]:
create_bucket('open-data-analytics-taxi-trips-bl')

'Created or already exists open-data-analytics-taxi-trips-bl bucket.'

In [6]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [ ]:
list_buckets(match='open')

In [8]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [9]:
list_bucket_contents(bucket='afsis', match='.csv', size_mb=250)

2009-2013/Dry_Chemistry/ICRAF/Bruker_TXRF/TXRF.csv (  0MB)
2009-2013/Georeferences/georeferences.csv (  0MB)
2009-2013/Wet_Chemistry/CROPNUTS/Wet_Chemistry_CROPNUTS.csv (  0MB)
2009-2013/Wet_Chemistry/ICRAF/Wet_Chemistry_ICRAF.csv (  1MB)
2009-2013/Wet_Chemistry/RRES/Wet_Chemistry_RRES.csv (  0MB)
tansis/Dry_Chemistry/ICRAF/Bruker_TXRF/TXRF.csv (  0MB)
tansis/Georeferences/georeferences.csv (  1MB)
tansis/Wet_Chemistry/CROPNUTS/Wet_Chemistry_CROPNUTS.csv (  0MB)
Matched file size is 0.0GB with 8 files
Bucket afsis total size is 4.8GB with 33093 files


In [ ]:
def preview_csv_dataset(bucket, key, rows=10):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url, nrows=rows)
    return data

In [10]:
df = preview_csv_dataset(bucket='afsis', key='2009-2013/Dry_Chemistry/ICRAF/Bruker_TXRF/TXRF.csv', rows=100)

NameError: name 'preview_csv_dataset' is not defined